# Bayesian Model

A simplistic Bayesian unigral LM.

In [ ]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')
import random

import beer
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

## Data

Generate some normally distributed data:

In [ ]:
voc_size = 10
X = torch.LongTensor(random.choices(range(voc_size), k=500))

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [ ]:
model = beer.UnigramLM.create(voc_size)

## Variational Bayes Training 

In [ ]:
epochs = 10
lrate = 1.

optim = beer.BayesianModelCoordinateAscentOptimizer(model.mean_field_groups, 
                                                    lrate)

elbos = []
for epoch in range(epochs):
    optim.zero_grad()
    elbo = beer.evidence_lower_bound(model, X, datasize=len(X))
    elbo.natural_backward()
    elbos.append(float(elbo) / len(X))
    optim.step()

In [ ]:
# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
fig.line(range(len(elbos)), elbos)

show(fig)

In [ ]:
model.weights